<a href="https://colab.research.google.com/github/Jbaruz/tools/blob/main/pdf_to_md_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get update
!apt-get install -y poppler-utils
!pip install pdfplumber pdf2image


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [8]:
import pdfplumber
import os
from google.colab import files

def table_to_markdown(table):
    """
    Convert a list of lists (from pdfplumber's extract_tables) into a Markdown table string.
    The first row is treated as a header row.
    """
    if not table:
        return ""

    # Determine the max number of columns in the table
    num_cols = max(len(row) for row in table)
    md_table = ""

    # First row as header
    header = table[0]
    header += [""] * (num_cols - len(header))  # pad if needed
    md_table += "| " + " | ".join(header) + " |\n"
    md_table += "| " + " | ".join(["---"] * num_cols) + " |\n"

    # Remaining rows
    for row in table[1:]:
        row += [""] * (num_cols - len(row))
        md_table += "| " + " | ".join(row) + " |\n"

    return md_table

def pdf_to_markdown(pdf_path, output_md_path="output.md", image_output_dir="images"):
    """
    Converts a PDF to a Markdown file with:
      - text
      - tables (in Markdown)
      - extracted images
    Saves the images in 'image_output_dir' and the markdown in 'output_md_path'.
    """
    os.makedirs(image_output_dir, exist_ok=True)

    markdown_content = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):

            # 1. Extract Text
            text = page.extract_text()
            if text:
                markdown_content += f"\n## Page {page_number}\n\n"
                markdown_content += text + "\n\n"

            # 2. Extract Tables
            tables = page.extract_tables()
            for table_index, table in enumerate(tables, start=1):
                markdown_content += f"### Table {table_index} (Page {page_number})\n\n"
                markdown_content += table_to_markdown(table) + "\n\n"

            # 3. Extract Images
            for image_index, img in enumerate(page.images, start=1):
                x0, top, x1, bottom = (img["x0"], img["top"], img["x1"], img["bottom"])

                # Safely attempt to crop image; skip if out of page bounds
                try:
                    crop = page.crop((x0, top, x1, bottom))
                    pil_img = crop.to_image(resolution=150).original
                except ValueError:
                    print(f"Skipping out-of-bounds image on page {page_number}, index {image_index}")
                    continue

                # Save the extracted image
                img_filename = f"page_{page_number}_img_{image_index}.png"
                img_path = os.path.join(image_output_dir, img_filename)
                pil_img.save(img_path)

                # Add a reference to this image in the Markdown
                markdown_content += f"![Page {page_number} - Image {image_index}]({os.path.join(image_output_dir, img_filename)})\n\n"

    # Write final Markdown file
    with open(output_md_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)

    print(f"Conversion complete! Markdown saved to: {output_md_path}")
    print(f"Images saved in folder: {image_output_dir}")


In [9]:
uploaded = files.upload()


Saving AI-new book.pdf to AI-new book (1).pdf


In [11]:
pdf_filename = list(uploaded.keys())[0]  # get the uploaded PDF file name
pdf_to_markdown(pdf_filename, output_md_path="output.md", image_output_dir="images")


Skipping out-of-bounds image on page 2, index 1
Skipping out-of-bounds image on page 194, index 1
Skipping out-of-bounds image on page 1711, index 1
Skipping out-of-bounds image on page 1712, index 1
Conversion complete! Markdown saved to: output.md
Images saved in folder: images


In [13]:
# Download the Markdown file
files.download("output.md")

# Zip and download the images folder
images_dir = "images"
zip_filename = "images.zip"
os.system(f"zip -r {zip_filename} {images_dir}")  # compress the images directory
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>